# Progression: Ordered Sequences for Workflows

**Progression** is an ordered sequence of UUIDs with Element identity. It combines:

- List-like operations: `append`, `extend`, `insert`, `remove`, `pop`
- Workflow operations: `move`, `swap`, `reverse`
- Idempotent set-like operations: `include`, `exclude`
- Query operations: `len`, `contains`, `getitem`, `index`

Progressions are primitives for state machine construction, representing execution order in workflow systems.

In [1]:
# Setup
from uuid import uuid4

from lionherd_core.base import Element, Progression

## 1. Construction & Append/Extend

Create progressions with UUIDs or Elements, then add items using `append` or `extend`.

In [2]:
# Empty progression
prog = Progression(name="task_queue")
print(f"Empty: {prog}")

# With UUIDs
tasks = [uuid4() for _ in range(3)]
prog_tasks = Progression(order=tasks, name="pending")
print(f"\nWith UUIDs: {prog_tasks}")

# With Elements (auto-converts to UUIDs)
elements = [Element() for _ in range(2)]
prog_elements = Progression(order=elements, name="active")
print(f"With Elements: {prog_elements}")

# Append and extend
prog.append(uuid4())
prog.extend([uuid4(), uuid4()])
print(f"\nAfter append/extend: {prog}, len={len(prog)}")

Empty: id=UUID('397fb3c9-3ed4-46db-be12-5adb4e3f6d79') created_at=datetime.datetime(2025, 11, 9, 2, 34, 37, 491901, tzinfo=datetime.timezone.utc) metadata={} name='task_queue' order=[]

With UUIDs: id=UUID('edc7c0fb-0944-4f90-8409-b6b46086ca76') created_at=datetime.datetime(2025, 11, 9, 2, 34, 37, 492310, tzinfo=datetime.timezone.utc) metadata={} name='pending' order=[UUID('45e5cf12-5a7b-4f25-9f37-98b674af08fa'), UUID('030624cc-706e-4ad0-8cd6-6f0977ae8c24'), UUID('fb31c278-41fa-438e-a31d-d07faa0703dd')]
With Elements: id=UUID('219135c3-d58c-4d47-a774-ea3fd83983a8') created_at=datetime.datetime(2025, 11, 9, 2, 34, 37, 492468, tzinfo=datetime.timezone.utc) metadata={} name='active' order=[UUID('56c5400f-0105-4aa9-9cc6-c86ffcf885d6'), UUID('bae47cc6-eb7a-40bb-8c09-9b24721ef45d')]

After append/extend: id=UUID('397fb3c9-3ed4-46db-be12-5adb4e3f6d79') created_at=datetime.datetime(2025, 11, 9, 2, 34, 37, 491901, tzinfo=datetime.timezone.utc) metadata={} name='task_queue' order=[UUID('0efb3f99

## 2. Query Operations

Check length, membership, and access items by index.

In [3]:
uids = [uuid4() for _ in range(5)]
prog = Progression(order=uids, name="execution_order")

# Length and membership
print(f"Length: {len(prog)}")
print(f"First UUID in progression: {uids[0] in prog}")
print(f"Random UUID in progression: {uuid4() in prog}")

# Index access (supports negative indices)
print(f"\nFirst item: {prog[0]}")
print(f"Last item: {prog[-1]}")
print(f"Slice [1:3]: {prog[1:3]}")

# Find index
print(f"\nIndex of third UUID: {prog.index(uids[2])}")

# Iteration
print(f"\nAll items: {list(prog)}")

Length: 5
First UUID in progression: True
Random UUID in progression: False

First item: e15bedde-1b2b-4b11-9a29-2ce0a9128447
Last item: eec68e31-96bb-490d-a7d9-e5e211b28695
Slice [1:3]: [UUID('5e079aa0-8bae-4019-a63b-d9a0524b1dff'), UUID('c52d8c76-c62e-49aa-9e80-c945079c61d0')]

Index of third UUID: 2

All items: [UUID('e15bedde-1b2b-4b11-9a29-2ce0a9128447'), UUID('5e079aa0-8bae-4019-a63b-d9a0524b1dff'), UUID('c52d8c76-c62e-49aa-9e80-c945079c61d0'), UUID('5fdf4bd8-c54c-4d5a-b949-c3576720b098'), UUID('eec68e31-96bb-490d-a7d9-e5e211b28695')]


## 3. Workflow Operations: Move & Swap

Reorder items for priority scheduling or state transitions.

In [4]:
# Create progression representing task priority
task1, task2, task3, task4 = uuid4(), uuid4(), uuid4(), uuid4()
prog = Progression(order=[task1, task2, task3, task4], name="priority_queue")

print(f"Initial order: {[str(u)[:8] for u in prog.order]}")

# Move task3 (index 2) to front (index 0) - high priority
prog.move(2, 0)
print(f"After move(2, 0): {[str(u)[:8] for u in prog.order]}")

# Swap first and last tasks
prog.swap(0, -1)
print(f"After swap(0, -1): {[str(u)[:8] for u in prog.order]}")

# Reverse entire queue
prog.reverse()
print(f"After reverse(): {[str(u)[:8] for u in prog.order]}")

Initial order: ['98f6bb3c', '0b1a9b15', '5ac2ba16', '6e850738']
After move(2, 0): ['5ac2ba16', '98f6bb3c', '0b1a9b15', '6e850738']
After swap(0, -1): ['6e850738', '98f6bb3c', '0b1a9b15', '5ac2ba16']
After reverse(): ['5ac2ba16', '0b1a9b15', '98f6bb3c', '6e850738']


## 4. Idempotent Operations: Include & Exclude

Set-like operations that are safe for retries and concurrent calls.

In [5]:
prog = Progression(name="task_registry")
task_id = uuid4()

# include: add if not present (idempotent)
print(f"First include: {prog.include(task_id)}")
print(f"Second include (duplicate): {prog.include(task_id)}")
print(f"Length after two includes: {len(prog)}")

# exclude: remove if present (idempotent)
print(f"\nFirst exclude: {prog.exclude(task_id)}")
print(f"Second exclude (absent): {prog.exclude(task_id)}")
print(f"Length after two excludes: {len(prog)}")

# Contrast with append (NOT idempotent)
prog.append(task_id)
prog.append(task_id)  # Creates duplicate
print(f"\nAfter two appends: len={len(prog)} (allows duplicates)")

First include: True
Second include (duplicate): False
Length after two includes: 1

First exclude: True
Second exclude (absent): False
Length after two excludes: 0

After two appends: len=2 (allows duplicates)


## 5. List Operations: Insert, Remove, Pop

Standard list operations with Element/UUID support.

In [6]:
uid1, uid2, uid3 = uuid4(), uuid4(), uuid4()
prog = Progression(order=[uid1, uid3], name="tasks")

# Insert at position
prog.insert(1, uid2)
print(f"After insert(1, uid2): {[str(u)[:8] for u in prog.order]}")

# Remove specific item
prog.remove(uid2)
print(f"After remove(uid2): {[str(u)[:8] for u in prog.order]}")

# Pop last item
popped = prog.pop()
print(f"Popped last: {str(popped)[:8]}")
print(f"Remaining: {[str(u)[:8] for u in prog.order]}")

# Pop first item (queue behavior)
prog.extend([uuid4(), uuid4()])
first = prog.popleft()
print(f"\nPopped first: {str(first)[:8]}")
print(f"After popleft: {len(prog)} items remaining")

After insert(1, uid2): ['0da78e23', '362450ed', '8f90c848']
After remove(uid2): ['0da78e23', '8f90c848']
Popped last: 8f90c848
Remaining: ['0da78e23']

Popped first: 0da78e23
After popleft: 2 items remaining


## 6. Serialization

Progressions inherit Element serialization with JSON support.

In [7]:
# Create progression
uids = [uuid4() for _ in range(3)]
original = Progression(order=uids, name="workflow_steps")

# Serialize to dict (JSON mode - UUIDs as strings)
data = original.to_dict(mode="json")
print("Serialized fields:")
print(f"  name: {data['name']}")
print(f"  order (first UUID): {data['order'][0][:8]}...")
print(f"  id: {data['id'][:8]}...")

# Deserialize (roundtrip)
restored = Progression.from_dict(data)
print(f"\nRestored: {restored}")
print(f"Order preserved: {restored.order == original.order}")
print(f"ID preserved: {restored.id == original.id}")

Serialized fields:
  name: workflow_steps
  order (first UUID): e41b5b84...
  id: bd87d0f2...

Restored: id=UUID('bd87d0f2-392e-4594-a8dd-7331c3882631') created_at=datetime.datetime(2025, 11, 9, 2, 34, 37, 516634, tzinfo=datetime.timezone.utc) metadata={} name='workflow_steps' order=[UUID('e41b5b84-6826-4ca7-86cb-34bb9c85dce0'), UUID('74216d6a-50b7-4e01-8e5b-426b81cdff51'), UUID('4992d46f-1ed7-4ce9-ac27-bdb6b5433462')]
Order preserved: True
ID preserved: True


## 7. Workflow State Machine Example

Progressions enable state machines for task execution tracking.

In [8]:
# State machine with three progressions
pending = Progression(name="pending")
active = Progression(name="active")
completed = Progression(name="completed")

# Initialize with tasks
tasks = [uuid4() for _ in range(5)]
pending.extend(tasks)

print(
    f"Initial state - pending: {len(pending)}, active: {len(active)}, completed: {len(completed)}"
)

# Start first task (pending → active)
task = pending.popleft()
active.append(task)
print(f"After start - pending: {len(pending)}, active: {len(active)}, completed: {len(completed)}")

# Complete task (active → completed)
task = active.pop()
completed.append(task)
print(
    f"After complete - pending: {len(pending)}, active: {len(active)}, completed: {len(completed)}"
)

# Retry failed task (active → pending)
pending.extend(active.order)
active.clear()
print(f"After retry - pending: {len(pending)}, active: {len(active)}, completed: {len(completed)}")

print(f"\nCompleted order preserves execution history: {[str(u)[:8] for u in completed.order]}")

Initial state - pending: 5, active: 0, completed: 0
After start - pending: 4, active: 1, completed: 0
After complete - pending: 4, active: 0, completed: 1
After retry - pending: 4, active: 0, completed: 1

Completed order preserves execution history: ['9057ebef']


## Summary

**Progression** provides ordered sequence management for workflows:

**Core Operations:**
- Construction: `Progression(order=[...], name="...")`
- List ops: `append`, `extend`, `insert`, `remove`, `pop`, `popleft`, `clear`
- Queries: `len`, `in`, `[index]`, `index()`, iteration

**Workflow Operations:**
- Reordering: `move(from, to)`, `swap(i, j)`, `reverse()`
- Idempotent: `include(item)`, `exclude(item)`

**Key Properties:**
- Element inheritance (id, created_at, metadata)
- UUID or Element input (auto-converts)
- Duplicates allowed (list semantics)
- Serialization support (JSON roundtrip)
- Workflow primitives for state machines

**Use Cases:**
- Task queues and execution order
- State machine construction (pending/active/completed)
- Priority scheduling with move/swap
- Idempotent registration with include/exclude